# Training the neural network
The scraping of news happens once a day, every day at the same time. After the news are scraped, every day at the same time, new NN is trained every day based on more info we have. This NN is saved into file and latter loaded when needed.
To make this work, the whole code is packed into functions because scheduler calls a function every day at the same time. The variables need to be global so all the functions can see them. The first cell explains the variables.

The files used for storing everything needed to reuse the NN again are:
* vectorizerOBJ for saving vetorizer and using it again with the user's news
* NNPickles.sav for saving the neural network
* part.txt saves some additional info about NN

In [1]:
documents = [] #this is an array of the texts from all the news
database = [] #this is an array which holds all the data about the news (id, source, title, text, date)
unreliableNUM=0 #the number of unreliable news
reliableNUM=0 #the number of reliable news
y= [] #the array which holds the info wether a news is reliable or not
adapted_vectors = [] #the array for vectors (matrix) from TFIDF

### First, load the data from db files

Get the relative path of the files because they are in different folders. This code is not in function because once the script is running, the path won't be changed (if the location of the project needs to be changed, the kernel would restart and execute this part of code again)

In [3]:
import csv
import sys
import os

mainDir = os.path.dirname(os.path.abspath(""))#the main folder which contains other folders
unreliablePath = os.path.join(mainDir, "UnreliableNews\\unreliable.csv")
#the same for reliable
reliablePath=os.path.join(mainDir, "ReliableNews\\reliable.csv")
print(unreliablePath, reliablePath)

C:\Users\MarijaStankovic\practica externa git\Practicas\UnreliableNews\unreliable.csv C:\Users\MarijaStankovic\practica externa git\Practicas\ReliableNews\reliable.csv


Opening the files and loading the data into arrays

In [4]:
def readDB():
    global documents
    global database
    global unreliableNUM
    global reliableNUM
    global y
    
    documents = []
    database = []
    unreliableNUM=0
    reliableNUM=0
    y= []
    
    with open(unreliablePath, 'r', encoding='utf8') as db:
        reader=csv.reader(db)
        next(reader, None) # skip the first line
        for row in reader:
            documents.append(row[4])
            database.append(row)
    unreliableNUM = len(documents)
    
    with open(reliablePath, 'r', encoding='utf8') as db:
        reader=csv.reader(db)
        next(reader, None) # skip the first line
        for row in reader:
            documents.append(row[4])
            database.append(row)
    reliableNUM = len(documents) - unreliableNUM
    
    for i in range(0, unreliableNUM):
        y.append(0)#unreliable are marked with 0
    for i in range(0, reliableNUM):
        y.append(1)#reliable are marked with 1

## Make the TFIDF vectors
first read the stop words

In [5]:
custom_stop_words = []
def stop_words_reader():
    with open( "stopwords.txt", "r" ) as fin:
        for line in fin.readlines():
            custom_stop_words.append( line.strip() )

and now make vectors and get the most important terms

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from sklearn import metrics
from numpy import asarray
from numpy import save
import pickle

def get_vectors():
    global adapted_vectors
    global array_of_indexes    
    adapted_vectors = []
    array_of_indexes=[]

    stop_words_reader()
    
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, min_df=20)    
    vectors = vectorizer.fit_transform(documents)
    adapted_vectors = vectors.toarray()
    pickle.dump(vectorizer, open(os.path.join(os.path.abspath(""), "NNfiles\\vectorizerOBJ.sav"), "wb"))  

## train NN
first normalize the vectors for NN and then create a classifier and save it

In [12]:
from sklearn.neural_network import MLPClassifier
import pickle
import Google

def NN():
    clf = MLPClassifier(hidden_layer_sizes=(100,20), learning_rate = 'constant', max_iter = 500)
    
    x=[]
    i=0
    for row in adapted_vectors:
        vector = []
        for j in row:
            vector.append(j)
        #google API for the title
#         vector.append(Google.google_API(database[i][1]))
        x.append(vector)
        i=i+1
        
    clf.fit(x, y)
    pickle.dump(clf, open(os.path.join(os.path.abspath(""), "NNfiles\\NNPickles.sav"), 'wb'))
    with open(os.path.join(os.path.abspath(""), "NNfiles\\part.txt"), "w") as p:
        p.writeline("Y")

In [10]:
def the_making():
    readDB()
    get_vectors()
    NN()

In [13]:
NN()

AttributeError: '_io.TextIOWrapper' object has no attribute 'writeline'

In [14]:
with open(os.path.join(os.path.abspath(""), "NNfiles\\part.txt"), "w") as p:
    p.write("Y")

In [11]:
the_making()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [9]:
# import schedule
# import time

# schedule.every().day.at("01:30").do(the_making)
# while 1:
#     schedule.run_pending()
#     time.sleep(1)